## Road Signs Bounding Box Prediction using Yolo

## 1. Load Data

In [1]:
from pathlib import Path
# set the path
images_path = Path("Data/images")
anno_path = Path("Data/annotations")
images_path, anno_path

(PosixPath('Data/images'), PosixPath('Data/annotations'))

In [2]:
# get a list of image files
image_path_list = list(images_path.glob("*.png"))
image_path_list[:5]

[PosixPath('Data/images/road289.png'),
 PosixPath('Data/images/road504.png'),
 PosixPath('Data/images/road262.png'),
 PosixPath('Data/images/road276.png'),
 PosixPath('Data/images/road510.png')]

In [3]:
# get a list of .xml files
anno_path_list = list(anno_path.glob("*.xml"))
anno_path_list[:5]

[PosixPath('Data/annotations/road712.xml'),
 PosixPath('Data/annotations/road706.xml'),
 PosixPath('Data/annotations/road289.xml'),
 PosixPath('Data/annotations/road538.xml'),
 PosixPath('Data/annotations/road510.xml')]

### 1.1 Check the info of the bounding box

In [4]:
# trying to load the annotation information
import xml.etree.ElementTree as ET

filepath = Path("Data/annotations/road207.xml")
# pick a single example
print(f"Name of .xml file: {filepath}")
root = ET.parse(filepath).getroot()
print("Corresponding image file:", Path(str(images_path) + '/'+ root.find("./filename").text))
print("Image width:", root.find("./size/width").text)
print("Image height:", root.find("./size/height").text)

for obj in root.iter('object'):
    print("Object name:", obj.find("name").text)
    print("xmin:", obj.find("bndbox/xmin").text)
    print("ymin:", obj.find("bndbox/ymin").text)

Name of .xml file: Data/annotations/road207.xml
Corresponding image file: Data/images/road207.png
Image width: 300
Image height: 400
Object name: speedlimit
xmin: 176
ymin: 184
Object name: crosswalk
xmin: 33
ymin: 214


### 1.2 Label Encode the class names

In [ ]:
# label encode the class names
class_dict = {
    "speedlimit" : 0,
    "stop": 1,
    "crosswalk": 2,
    "trafficlight": 3
}